<a href="https://colab.research.google.com/github/Flychuban/Malaria-Prediction-/blob/main/MalariaCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import os


import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dropout, Flatten, Dense, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [22]:
num_imgs = len(os.listdir('/content/drive/MyDrive/MalariaPrediction/cell_images/Test/Parasitized'))
num_imgs

2856

In [14]:
TARGET_SIZE = [128, 128]

In [24]:
datagen = ImageDataGenerator(rescale=1/255.0)

In [25]:
train_data = datagen.flow_from_directory("/content/drive/MyDrive/MalariaPrediction/cell_images/Train",
                                         target_size=TARGET_SIZE,
                                         class_mode = 'binary',
                                         batch_size = 16,
                                         subset='training')

Found 21846 images belonging to 2 classes.


In [28]:
val_data = datagen.flow_from_directory('/content/drive/MyDrive/MalariaPrediction/cell_images/Test',
                                       target_size=TARGET_SIZE,
                                       class_mode='binary',
                                       batch_size=16)

Found 5712 images belonging to 2 classes.


Build model

In [31]:
model = Sequential()

In [33]:
model.add(Conv2D(16, (3,3), activation='relu', input_shape=(128, 128, 3)))
model.add(MaxPool2D(2, 2))
model.add(Dropout(0.2))

model.add(Conv2D(32, (3,3), activation='relu'))
model.add(MaxPool2D(2, 2))
model.add(Dropout(0.3))

model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPool2D(2,2))
model.add(Dropout(0.3))

model.add(Flatten())

model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))